In [1]:
import mindpype as mp

import numpy as np
from random import shuffle

In [2]:
# create a session and a graph
session = mp.Session.create()
trial_graph = mp.Graph.create(session)

In [3]:
# Create random initialization (training) data and labels
training_data = np.random.random((120,12,500))   
labels = np.asarray([0]*60 + [1]*60)


# Create tensors from the data and labels
init_data = mp.Tensor.create_from_data(session,training_data.shape,training_data)
init_labels = mp.Tensor.create_from_data(session,labels.shape,labels)


In [4]:
# Create an input tensor with dummy data
input_tensor = mp.Tensor.create_from_data(session, (12, 500), np.random.randn(12, 500))

# Create a scalar that will be populated with the classifier label
classifier_label = mp.Scalar.create_from_value(session,-1)

# Create intermediate (virtual) tensors for the intermediate steps of the pipeline
intermediate_tensors = [mp.Tensor.create_virtual(session), 
                        mp.Tensor.create_virtual(session),
                        mp.Tensor.create_virtual(session),
                        mp.Tensor.create_virtual(session)]

In [5]:
# create filter parameters
order = 4
bandpass = (8,35) # in Hz
fs = 250

# Create a filter object using the parameters 
filter_obj = mp.Filter.create_butter(session,order,bandpass,btype='bandpass',fs=fs,implementation='sos')

In [6]:
# Create a classifier object
classifier = mp.Classifier.create_LDA(session)

In [7]:
# add the processing nodes to the graph using the factory methods
filter_kernel = mp.kernels.FilterKernel.add_filter_node(trial_graph,input_tensor,filter_obj,intermediate_tensors[0], axis = 1)

CSP_kernel = mp.kernels.CommonSpatialPatternKernel.add_uninitialized_CSP_node(trial_graph, intermediate_tensors[0], intermediate_tensors[1], init_data, init_labels, 2)
var_kernel = mp.kernels.VarKernel.add_var_node(trial_graph, intermediate_tensors[1], intermediate_tensors[2], axis = 1)
log_kernel = mp.kernels.LogKernel.add_log_node(trial_graph, intermediate_tensors[2], intermediate_tensors[3], axis = 1)
LDA_kernel = mp.kernels.ClassifierKernel.add_classifier_node(trial_graph, intermediate_tensors[4], classifier, classifier_label, None, None)

AttributeError: type object 'VarKernel' has no attribute 'add_variance_node'

In [ ]:
# verify the session (i.e. schedule the nodes) and ensure the inputs and outputs are connected properly
try:
    trial_graph.verify()
except:
    print("Test Failed D=")

		FilterNode Scheduled
		CSPNode Scheduled
		ClassifierNode Scheduled
Test Failed D=


In [ ]:
# Since the graph contains nodes that must be initialzed/trained, we must call initialize() before running the graph
try:
    trial_graph.initialize()
except:
    print("Test Failed D=")

In [ ]:
# RUN!
trial_graph.execute()
# print the value of the most recent trial
print("Trial {}, Predicted label = {}\n".format(1, classifier_label.data))


print("Test Passed =D")

		FilterNode Scheduled
		CSPNode Scheduled
		ClassifierNode Scheduled


ValueError: X has 500 features per sample; expecting 1000
Test execution of node Classifier failed during verification. Please check parameters. - Node: Classifier failed verification